# Import modules

In [ ]:
# Data Handlers
import pandas as pd
import numpy as np
from PIL import Image
from PIL import ImageOps

# Pytorch
import torch
import torch.nn as nn  # NN; networks (CNN, RNN, losses)
import torch.optim as optim  # Optimizers (Adam, Adadelta, Adagrad)
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, IterableDataset  # Dataset manager

# Other
from tqdm import tqdm
import os

# Graphics
from matplotlib import pyplot as plt
import seaborn as sns

# Additional modules
from dataset_creator import generate_csv
from assistive_funcs import filtering_image, check_ssim, convert_to_grayscale
from math import floor

sns.set()

# Define constants

In [ ]:
datasets_path = r"..\datasets\csv_files"
img_path = r"..\datasets\images"
models_path = r"..\models"

# Generate Dataset

In [ ]:
create_dataset = False
if create_dataset:
    generate_csv(win_size=13, dump_to_file=5000, step=4)

# Create DataLoader

In [ ]:
""" Will be done in the future """
# class CustomDataLoader:
#     def __init__(self, datasets_path, dataset_name, batch_size, split_size, state) -> None:
#         self.datasets_path = datasets_path
#         self.dataset_name = dataset_name
#         self.main_path = f"{datasets_path}\{dataset_name}"
        
#         self.batch_size = batch_size
#         self.split_size = split_size
        
#         self.test_batches, self.real_test_samples = self.get_test_amount()
#         self.skip_rows = self.real_test_samples
        
#         self.state = state
#         self.counter = 1

#     def __iter__(self):
#         if self.state == "Train":
#             self.train = pd.read_csv(self.main_path, chunksize=self.batch_size,
#                                     header=None, index_col=None, iterator=True)
#             return self.train
#         else:
#             self.test = pd.read_csv(self.main_path, chunksize=self.batch_size,
#                                     header=None, index_col=None, iterator=True,
#                                     skiprows=self.skip_rows)
#             return self.test

#     def __next__(self):
#         print(f"{self.state = }, {self.test_batches = }")
#         if self.state == "Train":
#             self.counter += 1
#             if self.counter >= self.test_batches:
#                 print("StopIteration")
#                 raise StopIteration
            
#             return self.chunk.get_chunk()
#         return self.chunk.get_chunk()
        

#     def get_len_data(self ) -> int:
#         idx_start = self.dataset_name.find("L") + 1
#         idx_finish = self.dataset_name.find(".")
#         length = int(self.dataset_name[idx_start:idx_finish])
#         return length
    
#     def get_test_amount(self) -> tuple:
#         length = self.get_len_data()
#         test_smaples = int(length * self.split_size)
#         test_batches = floor(test_smaples / self.batch_size)
#         real_test_samples = test_batches * self.batch_size
#         print(f"{length = }, {test_batches = }, {real_test_samples = }")
#         return test_batches, real_test_samples
    
    
# train = CustomDataLoader(datasets_path=datasets_path,
#                      dataset_name="tmp_L6.csv",
#                      batch_size=2,
#                      split_size=0.5,
#                      state="Train")
# test = CustomDataLoader(datasets_path=datasets_path,
#                      dataset_name="tmp_L6.csv",
#                      batch_size=2,
#                      split_size=0.5,
#                      state="Test")
# for i in train:
#     display(i)
# print("=" * 20)
# for i in test:
#     display(i)


In [ ]:
class DefaultDL(Dataset):
    def __init__(self, dataset_path, dataset_name):
        super().__init__()
        main_path = f"{dataset_path}\{dataset_name}"
        self.data = pd.read_csv(main_path, header=None)

        target_ind = self.data.shape[1] - 1
        self.data.rename(columns={target_ind: "target"}, inplace=True)

        self.targets = self.data["target"]
        self.data.drop(columns="target", inplace=True)

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, index):
        x = torch.Tensor(self.data.iloc[index].to_numpy()).float()
        y = torch.Tensor([self.targets.iloc[index]]).float()
        return x, y

## Load dataset

In [ ]:
dataset_name = r"W13_S4_L231040.csv"
win_size = 13
batch_size = 128

In [ ]:
dataset = DefaultDL(datasets_path, dataset_name)

In [ ]:
train_size = int(dataset.__len__() * 0.95)
test_size = dataset.__len__() - train_size
print(f"{train_size = }\n{test_size = },\n{train_size + test_size = }")

In [ ]:
train_set, test_set = torch.utils.data.random_split(dataset, [train_size, test_size])
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)

# NN Model

In [ ]:
class DefaultModel(nn.Module):
    def __init__(self, in_len, out_len) -> None:
        super().__init__()
        self.in_len = in_len
        self.out_len = out_len
        self.hid_n = 700
        
        self.fcs = nn.Sequential(
            nn.Linear(self.in_len, self.hid_n),
            nn.BatchNorm1d(self.hid_n),
            nn.ReLU(),
            nn.Linear(self.hid_n, self.hid_n * 2),
            nn.BatchNorm1d(self.hid_n * 2),
            nn.ReLU(),
            nn.Linear(self.hid_n * 2, self.hid_n * 3),
            nn.BatchNorm1d(self.hid_n * 3),
            nn.ReLU(),
            
            nn.Linear(self.hid_n * 3, self.hid_n * 3),
            nn.BatchNorm1d(self.hid_n * 3),
            nn.LeakyReLU(),
            nn.Linear(self.hid_n * 3, self.hid_n * 3),
            nn.BatchNorm1d(self.hid_n * 3),
            nn.LeakyReLU(),
            
            nn.Linear(self.hid_n * 3, self.hid_n * 2),
            nn.BatchNorm1d(self.hid_n * 2),
            nn.ReLU(),
            nn.Linear(self.hid_n * 2, self.hid_n),
            nn.BatchNorm1d(self.hid_n),
            nn.ReLU(),
            nn.Linear(self.hid_n, self.out_len),
        )

    def forward(self, x):
        x = self.fcs(x)
        return x

## Define NN's constants

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
learning_rate = 0.003
num_epoches = 1
display(device)

## Initialize model

In [ ]:
model = DefaultModel(in_len=(win_size ** 2), out_len=1).to(device=device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
losses = []
losses_append = losses.append

valid_losses = []
valid_losses_append = valid_losses.append

for epoch in range(num_epoches):
    model.train()
    for batch_ind, (data, targets) in tqdm(enumerate(train_loader)):
        # Data on cuda
        data = data.to(device=device)
        targets = targets.to(device=device)
        
        # Forward
        scores = model(data) # Equal to model.forward(data)
        loss = criterion(scores, targets)
        if batch_ind % 3 == 0:
            losses_append(loss.item())
        # Backprop
        optimizer.zero_grad()
        loss.backward()

        # Gradient descent or adam step
        optimizer.step()
    model.eval()
    with torch.no_grad():
        for batch_ind, (data, targets) in tqdm(enumerate(test_loader)):
            data = data.to(device=device)
            targets = targets.to(device=device)
            
            # Forward
            scores = model(data) # Equal to model.forward(data)
            loss = criterion(scores, targets)
            valid_losses_append(loss.item())



In [ ]:
sns.set(rc={"figure.figsize": (7, 6)})
plt.plot(losses);

In [ ]:
sns.set(rc={"figure.figsize": (7, 6)})
plt.plot(valid_losses);

In [ ]:
torch.save(model, f"{models_path}\{dataset_name}")

In [ ]:
del dataset
del train_loader
del test_loader
del train_set
del test_set
del data

# Check NN works

In [ ]:
path_to_noised_imgs = r"D:\Projects\PythonProjects\NIR\datasets\imgs_with_noise"

In [ ]:
for i in range(1, 11):   
    filtering_image(model, path_to_noised_imgs, f"{i}.jpg", 13, device)

In [ ]:
filtered_images = r"D:\Projects\PythonProjects\NIR\datasets\filtered_imgs"
# convert_to_grayscale(genuine_images)


In [ ]:

genuine_images = r"D:\Projects\PythonProjects\NIR\datasets\gray_images"

check_ssim(filtered_images, genuine_images)